---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    graphIdentification = []
    
    for index in range(len(P1_Graphs)):
        clusters = nx.average_clustering(P1_Graphs[index])
        shortPathLength = nx.average_shortest_path_length(P1_Graphs[index])
        degVals = sorted(set(P1_Graphs[index].degree().values()))
        degreeDist = [list(P1_Graphs[index].degree().values()).count(val)/float(nx.number_of_nodes(P1_Graphs[index])) for val in degVals]
        
        if len(degreeDist) > 10:
            graphIdentification.append("PA")
        elif clusters < 0.1:
            graphIdentification.append("SW_H")
        else:
            graphIdentification.append("SW_L")
            
    return graphIdentification
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
def salary_predictions():
    nodes = pd.DataFrame(index = G.nodes())
    
    nodes["Dept"] = pd.Series(nx.get_node_attributes(G, "Department"))
    nodes["SalOfManagement"] = pd.Series(nx.get_node_attributes(G, "ManagementSalary"))
    nodes["Deg"] = pd.Series(nx.degree(G))
    nodes["Clustered"] = pd.Series(nx.clustering(G))
    nodes["Center"] = pd.Series(nx.degree_centrality(G))
    nodes["Closeness"] = pd.Series(y for x, y in nx.closeness_centrality(G).items())
    nodes["Betweeness"] = pd.Series(nx.betweenness_centrality(G))
    
    nx.clustering(G)
    
    X_test = nodes.loc[nodes["SalOfManagement"].isnull(), ['Dept', 'Deg', 'Clustered', 'Center', 'Betweeness', 'Closeness']]
    X_train = nodes.loc[~nodes["SalOfManagement"].isnull(), ['Dept', 'Deg', 'Clustered', 'Center', 'Betweeness', 'Closeness']]
    y_train = nodes.loc[~nodes["SalOfManagement"].isnull(), ["SalOfManagement"]]
    
    gradBoostClf = GradientBoostingClassifier(n_estimators = 600, learning_rate = 0.1, max_depth = 5, random_state = 0).fit(X_train, y_train)
    
    return pd.Series(gradBoostClf.predict(X_test), index = X_test.index.values)

salary_predictions()

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1       0.0
2       1.0
5       1.0
8       0.0
14      0.0
18      0.0
27      0.0
30      1.0
31      0.0
34      0.0
37      0.0
40      0.0
45      0.0
54      0.0
55      1.0
60      0.0
62      1.0
65      1.0
77      0.0
79      0.0
97      0.0
101     0.0
103     1.0
108     0.0
113     0.0
122     0.0
141     0.0
142     1.0
144     0.0
145     1.0
       ... 
913     0.0
914     0.0
915     0.0
918     0.0
923     0.0
926     0.0
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.0
950     0.0
951     0.0
953     0.0
959     0.0
962     0.0
963     0.0
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler


def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']

    scaler = MinMaxScaler()
    futConDF = pd.DataFrame(index=[(x[0], x[1]) for x in list(nx.preferential_attachment(G))])
    futConDF['prefAttach'] = [x[2] for x in list(nx.preferential_attachment(G))]
    
    futConDF['jaccard'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    
    soundHopCN = pd.DataFrame(index=[(x[0], x[1]) for x in list(nx.cn_soundarajan_hopcroft(G))])
    soundHopCN['soundHopCN'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G))]
    futConDF = futConDF.join(soundHopCN, how = "outer")
    futConDF['soundHopCN'] = futConDF['soundHopCN'].fillna(value=0)
    futConDF['soundHopRA'] = [x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G))]
    futConDF = future_connections.join(futConDF, how = "outer")

    futConDFTrain = futConDF[~pd.isnull(futConDF['Future Connection'])]
    futConDFTest = futConDF[pd.isnull(futConDF['Future Connection'])]
    
    X_train = futConDFTrain[['prefAttach', 'jaccard', 'soundHopCN', 'soundHopRA']]
    y_train = futConDFTrain['Future Connection']
    
    X_test = futConDFTest[['prefAttach', 'jaccard', 'soundHopCN', 'soundHopRA']]
    
    X_trainNormalized = scaler.fit_transform(X_train)
    X_testNormalized = scaler.transform(X_test)
    
    clf = MLPClassifier(hidden_layer_sizes=[10, 5], alpha=5, random_state=0, solver='lbfgs')
    clf.fit(X_trainNormalized, y_train)
    
    predVals = clf.predict_proba(X_testNormalized)[:, 1]
    predValsSeries = pd.Series(predVals, X_test.index)
    
    final = future_connections[pd.isnull(future_connections['Future Connection'])]
    final['predictions'] = [predValsSeries[x] for x in final.index]
    
    return final['predictions']


new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.026874
(542, 751)    0.010587
(20, 426)     0.585340
(50, 989)     0.010646
(942, 986)    0.010709
(324, 857)    0.010638
(13, 710)     0.143431
(19, 271)     0.135196
(319, 878)    0.010666
(659, 707)    0.010593
(49, 843)     0.010685
(208, 893)    0.010607
(377, 469)    0.008322
(405, 999)    0.017850
(129, 740)    0.016789
(292, 618)    0.018181
(239, 689)    0.010670
(359, 373)    0.009226
(53, 523)     0.182860
(276, 984)    0.010682
(202, 997)    0.010694
(604, 619)    0.056614
(270, 911)    0.010669
(261, 481)    0.071278
(200, 450)    0.995157
(213, 634)    0.010569
(644, 735)    0.044511
(346, 553)    0.010422
(521, 738)    0.010141
(422, 953)    0.017621
                ...   
(672, 848)    0.010669
(28, 127)     0.932738
(202, 661)    0.010397
(54, 195)     0.992417
(295, 864)    0.010629
(814, 936)    0.010578
(839, 874)    0.010709
(139, 843)    0.010634
(461, 544)    0.009945
(68, 487)     0.009911
(622, 932)    0.010614
(504, 936)    0.016687
(479, 528) 